In [ ]:
import yfinance as yf
from prophet import Prophet
import pandas as pd

In [ ]:
# ステップ1：Yahoo Financeから過去の株価データを取得
ticker = "NVDA"
data = yf.download(ticker, start="2020-02-01", end="2025-02-21")

# データの前処理、Date, Open, High, Low, Close, Volume列を保持
data = data.reset_index()[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]

# MultiIndex の列名がある場合、解除する
if isinstance(data.columns, pd.MultiIndex):
    data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

# Volume の予測用データフレーム作成
df_volume = data[['Date', 'Volume']].rename(columns={'Date': 'ds', 'Volume': 'y'})

# Prophet の入力データ形式に合わせる
df_volume['ds'] = pd.to_datetime(df_volume['ds']).dt.tz_localize(None)  # 日付列のタイムゾーンを削除
df_volume['y'] = pd.to_numeric(df_volume['y'], errors='coerce')  # 数値以外を NaN に変換
df_volume = df_volume.dropna()  # NaN を含む行を削除

# Volume の予測を行う Prophet モデルを作成
model_volume = Prophet(daily_seasonality=False)
model_volume.fit(df_volume)

# 未来の Volume を予測
future_volume = model_volume.make_future_dataframe(periods=1, freq='B')  # 1 営業日の未来を予測
forecast_volume = model_volume.predict(future_volume)

# 未来の Volume の予測結果をデータに追加
data['Volume_pred'] = forecast_volume['yhat'].iloc[:len(data)].values  # 過去分のみ追加

# ステップ2：Open, High, Low, Close の予測用データフレーム作成
df_open = data[['Date', 'Open', 'Close', 'High', 'Low', 'Volume_pred']].rename(columns={'Date': 'ds', 'Open': 'y'})
df_high = data[['Date', 'High', 'Open', 'Close', 'Low', 'Volume_pred']].rename(columns={'Date': 'ds', 'High': 'y'})
df_low = data[['Date', 'Low', 'Open', 'Close', 'High', 'Volume_pred']].rename(columns={'Date': 'ds', 'Low': 'y'})
df_close = data[['Date', 'Close', 'Open', 'High', 'Low', 'Volume_pred']].rename(columns={'Date': 'ds', 'Close': 'y'})


[*********************100%***********************]  1 of 1 completed
DEBUG:cmdstanpy:input tempfile: /tmp/tmpblpqz1er/ojk115wh.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpblpqz1er/c2afht5o.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=17539', 'data', 'file=/tmp/tmpblpqz1er/ojk115wh.json', 'init=/tmp/tmpblpqz1er/c2afht5o.json', 'output', 'file=/tmp/tmpblpqz1er/prophet_modelz9ps1epb/prophet_model-20241123101610.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:16:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:16:10 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
data

,Date,Open,High,Low,Close,Volume,Volume_pred
0,2020-11-23 00:00:00+00:00,13.171250,13.382500,13.015500,13.140000,361356000,2.424699e+08
1,2020-11-24 00:00:00+00:00,13.142500,13.162000,12.885000,12.957750,346500000,2.406128e+08
2,2020-11-25 00:00:00+00:00,13.032500,13.269500,13.020000,13.234750,259916000,2.534892e+08
3,2020-11-27 00:00:00+00:00,13.375000,13.407500,13.170500,13.261250,167276000,2.575676e+08
4,2020-11-30 00:00:00+00:00,13.265000,13.412500,12.972250,13.401500,286952000,2.460564e+08
...,...,...,...,...,...,...,...
1001,2024-11-15 00:00:00+00:00,144.869995,145.240005,140.080002,141.979996,250132900,3.020763e+08
1002,2024-11-18 00:00:00+00:00,139.500000,141.550003,137.149994,140.149994,221866000,2.912905e+08
1003,2024-11-19 00:00:00+00:00,141.320007,147.130005,140.990005,147.009995,227834900,2.888161e+08
1004,2024-11-20 00:00:00+00:00,147.410004,147.559998,142.729996,145.889999,309871700,3.011178e+08


In [ ]:
df_open['ds'] = pd.to_datetime(df_open['ds']).dt.tz_localize(None)
df_high['ds'] = pd.to_datetime(df_high['ds']).dt.tz_localize(None)
df_low['ds'] = pd.to_datetime(df_low['ds']).dt.tz_localize(None)
df_close['ds'] = pd.to_datetime(df_close['ds']).dt.tz_localize(None)

# 各データフレームの Prophet モデルを作成
model_open = Prophet(daily_seasonality=False)
model_open.add_regressor('Close')
model_open.add_regressor('High')
model_open.add_regressor('Low')
model_open.add_regressor('Volume_pred')
model_open.fit(df_open)

model_high = Prophet(daily_seasonality=False)
model_high.add_regressor('Open')
model_high.add_regressor('Close')
model_high.add_regressor('Low')
model_high.add_regressor('Volume_pred')
model_high.fit(df_high)

model_low = Prophet(daily_seasonality=False)
model_low.add_regressor('Open')
model_low.add_regressor('Close')
model_low.add_regressor('High')
model_low.add_regressor('Volume_pred')
model_low.fit(df_low)

model_close = Prophet(daily_seasonality=False)
model_close.add_regressor('Open')
model_close.add_regressor('High')
model_close.add_regressor('Low')
model_close.add_regressor('Volume_pred')
model_close.fit(df_close)

# 未来の値を予測
future = model_close.make_future_dataframe(periods=1, freq='B')  # 1 営業日の未来を予測
future['Open'] = data['Open'].iloc[-1]
future['High'] = data['High'].iloc[-1]
future['Low'] = data['Low'].iloc[-1]
future['Close'] = data['Close'].iloc[-1]
future['Volume_pred'] = forecast_volume['yhat'].iloc[-1]  # 未来の Volume の予測値を利用

# 各モデルで予測
forecast_open = model_open.predict(future)
forecast_high = model_high.predict(future)
forecast_low = model_low.predict(future)
forecast_close = model_close.predict(future)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpblpqz1er/xcqdiyto.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpblpqz1er/bshe1dwb.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=95089', 'data', 'file=/tmp/tmpblpqz1er/xcqdiyto.json', 'init=/tmp/tmpblpqz1er/bshe1dwb.json', 'output', 'file=/tmp/tmpblpqz1er/prophet_modell5dkdlny/prophet_model-20241123101610.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:16:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:16:11 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpblpqz1er/es2po8y8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpblpqz1er/g9lo4gmb.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

In [ ]:
# 予測結果を取得
predicted_open = forecast_open['yhat'].iloc[-1]
predicted_high = forecast_high['yhat'].iloc[-1]
predicted_low = forecast_low['yhat'].iloc[-1]
predicted_close = forecast_close['yhat'].iloc[-1]
predicted_volume = forecast_volume['yhat'].iloc[-1]

# 結果を出力
# print(f"予測される明日の始値 (Open): {predicted_open:.2f}")
# print(f"予測される明日の終値 (Close): {predicted_close:.2f}")
# print(f"予測される明日の最高値 (High): {predicted_high:.2f}")
# print(f"予測される明日の最安値 (Low): {predicted_low:.2f}")
# print(f"予測される明日の売買代金 (Volume): {predicted_volume:.2f}")
# print(f"ブルマーケットの確率: {probability_bull:.2%}")
# print(f"ベアマーケットの確率: {probability_bear:.2%}")

print(f"預測的明天的開盤價 (Open): {predicted_open:.2f}")
print(f"預測的明天的收盤價 (Close): {predicted_close:.2f}")
print(f"預測的明天的最高價 (High): {predicted_high:.2f}")
print(f"預測的明天的最低價 (Low): {predicted_low:.2f}")
print(f"預測的明天的成交量 (Volume):{predicted_volume:.2f}")


預測的明天的開盤價 (Open): 147.81
預測的明天的收盤價 (Close): 145.15
預測的明天的最高價 (High): 151.80
預測的明天的最低價 (Low): 144.19
預測的明天的成交量 (Volume):304226454.95
